TestTask

**** Calculate metrics from the given dataset

1) Revenue Contribution 

2) Total Orders 

3) Average 12 Month Contribution 

4) Average 12 Month Orders

5) Average Order Value 

6) Last 12 Month Contribution 

7) Last 12 Month Orders

8) Last Order Value 

9) % of Customers that has more value

10) % of Orders generated by client

11) % of Revenue generated by client

12) Time since last order in days


**** Create clients clusters (Best, Good, and Low) by metrics and Customer Type by Industry

In [1]:
# test start time 18:00 (GMT+3:00) 7/06/2021
# test end time 21:00 (GMT+3:00) 7/06/2021
# test start time 8:30 (GMT+3:00) 8/06/2021
# test end time 10:40 (GMT+3:00) 8/06/2021
# test start time 17:00 (GMT+3:00) 9/06/2021
# test end time 21:00 (GMT+3:00) 9/06/2021

In [2]:
import pandas as pd
from datetime import datetime, date, time
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel(r'Sample Line Items_long.xlsx')

In [4]:
# test df state
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14697 entries, 0 to 14696
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Internal ID                14697 non-null  int64         
 1   *                          14697 non-null  object        
 2   Date                       14697 non-null  datetime64[ns]
 3   Document Number            14697 non-null  object        
 4   Company Name               14697 non-null  object        
 5   Contact Name               14697 non-null  object        
 6   Account                    14697 non-null  object        
 7   Item                       14697 non-null  object        
 8   Memo                       14665 non-null  object        
 9   Amount                     14697 non-null  float64       
 10  Sales Rep                  14697 non-null  object        
 11  Class                      14697 non-null  object        
 12  Loca

In [5]:
df.shape

(14697, 14)

In [6]:
df.columns

Index(['Internal ID', '*', 'Date', 'Document Number', 'Company Name',
       'Contact Name', 'Account', 'Item', 'Memo', 'Amount', 'Sales Rep',
       'Class', 'Location', 'Customer Type by Industry'],
      dtype='object')

In [7]:
df.head(5)

,Internal ID,*,Date,Document Number,Company Name,Contact Name,Account,Item,Memo,Amount,Sales Rep,Class,Location,Customer Type by Industry
0,693551,,2021-01-04,TJAN210001,"Reynolds Parrino Shadwick, P.A.",Britney Mangum-Cabrera,40100 Sales-Legal Paper,"Blowback : Blowback B&W - 8.5""x11""","B&W Images Printed from File 8.5""x11""",101.88,Michael DeLessio,Paper,Tampa,Legal
1,693551,,2021-01-04,TJAN210001,"Reynolds Parrino Shadwick, P.A.",Britney Mangum-Cabrera,40100 Sales-Legal Paper,"Blowback : Blowback Color - 8.5""x11""","Color Images Printed from File 8.5""x11""",95.95,Michael DeLessio,Paper,Tampa,Legal
2,693552,,2021-01-04,TJAN210002,"Reynolds Parrino Shadwick, P.A.",Laura Craig,40100 Sales-Legal Paper,"Blowback : Blowback B&W - 8.5""x11""","B&W Images Printed from File 8.5""x11""",183.50,Michael DeLessio,Paper,Tampa,Legal
3,693894,,2021-01-04,RJAN210001,BVR Construction,Garland Beasley,40125 Sales-AEC Printing/Scanning,AEC Printing/Copying : Oversize Color (AEC) - ...,14 Oversize Color Line Drawings Copy/Print X ...,126.00,Rochester House,Paper,Rochester,AEC
4,693899,,2021-01-04,TJAN210003,Prugh & Associates,Sarah Wadsworth,40100 Sales-Legal Paper,"Blowback : Blowback Color - 8.5""x11""","Color Images Printed from File 8.5""x11""",10.00,Michael DeLessio,Paper,Tampa,Legal


----

### 1) Revenue Contribution

### 2) Total Orders 

In [8]:
#1) Revenue Contribution
print('Revenue Contribution -', df['Amount'].sum())

#2) Total Orders 
print('Total orders (unique funds transfer) -', df.shape[0])

Revenue Contribution - 5128193.74
Total orders (unique funds transfer) - 14697


In [9]:
# hypothesis test: one order/document (Document Number) - several services/products, several payments ('Amount') 
df[df['Document Number'].str.match('TJAN210001')] # for example

,Internal ID,*,Date,Document Number,Company Name,Contact Name,Account,Item,Memo,Amount,Sales Rep,Class,Location,Customer Type by Industry
0,693551,,2021-01-04,TJAN210001,"Reynolds Parrino Shadwick, P.A.",Britney Mangum-Cabrera,40100 Sales-Legal Paper,"Blowback : Blowback B&W - 8.5""x11""","B&W Images Printed from File 8.5""x11""",101.88,Michael DeLessio,Paper,Tampa,Legal
1,693551,,2021-01-04,TJAN210001,"Reynolds Parrino Shadwick, P.A.",Britney Mangum-Cabrera,40100 Sales-Legal Paper,"Blowback : Blowback Color - 8.5""x11""","Color Images Printed from File 8.5""x11""",95.95,Michael DeLessio,Paper,Tampa,Legal


In [10]:
#2b) Total Orders 
print('Total orders (by unique document numbers) -', df.groupby(['Document Number']).sum().reset_index().shape[0])

Total orders (by unique document numbers) - 7953


----

### 3) Average 12 Month Contribution
### 4) Average 12 Month Orders

In [11]:
# Check date range 
print(df['Date'].min())
print(df['Date'].max())

2014-02-16 00:00:00
2021-04-30 00:00:00


In [12]:
# As we seen we dont have 12 month period for calculating such metrics.
# Size? Сonditions of the problem don't point out which time interval exactly, so we calculate all dataset. 
df_12=df.groupby(pd.Grouper(key='Date', freq='Y'))['Amount'].agg({'count', 'sum'})
df_12.index=df_12.index.strftime("%Y")
df_12.rename(columns={'sum':'Total Contribution', 'count':'Order Count'}, inplace=True)
df_12.index.rename('Year', inplace=True)
df_12.reset_index(inplace=True)
df_12['AVG 12M Contribution']=df_12['Total Contribution'].apply(lambda x: round(x/12, 2))
df_12['AVG 12M Orders']=df_12['Order Count'].apply(lambda x: round(x/12, 1))
df_12

,Year,Order Count,Total Contribution,AVG 12M Contribution,AVG 12M Orders
0,2014,319,43412.63,3617.72,26.6
1,2015,365,63335.42,5277.95,30.4
2,2016,366,64485.76,5373.81,30.5
3,2017,365,149059.01,12421.58,30.4
4,2018,365,292881.44,24406.79,30.4
5,2019,365,367754.85,30646.24,30.4
6,2020,366,214979.75,17914.98,30.5
7,2021,12186,3932284.88,327690.41,1015.5


In [13]:
last_year = datetime.now().year-1
df_12_const = df_12.loc[df_12['Year'] == str(last_year),'AVG 12M Contribution']
df_12_orders = df_12.loc[df_12['Year'] == str(last_year),'AVG 12M Orders']
print('In', last_year, 'AVG 12M Contribution is', df_12_const.iloc[0])
print('In', last_year, 'AVG 12M Orders is', df_12_orders.iloc[0])

In 2020 AVG 12M Contribution is 17914.98
In 2020 AVG 12M Orders is 30.5


----

### 6) Last 12 Month Contribution
### 7) Last 12 Month Orders

In [14]:
# in future cases we can input anyinterval 
timeinterval_end = datetime.now()
timeinterval_start = timeinterval_end + pd.DateOffset(months=-12)

In [15]:
# df_last=df[(df['Date']<=timeinterval_start)&(df['Date']>=timeinterval_end)]
df_last=df[(df['Date']>=timeinterval_start)&(df['Date']<=timeinterval_end)]
df_last=df_last.groupby(pd.Grouper(key='Date', freq='M'))['Amount'].agg({'count', 'sum'})
df_last.rename(columns={'sum':'Monthly Contribution', 'count':'Monthly Orders Count'}, inplace=True)
#df_12.index.rename('Year', inplace=True)
#df_12.reset_index(inplace=True)
df_last

,Monthly Orders Count,Monthly Contribution
Date,,
2020-06-30,21,7463.05
2020-07-31,31,15094.10
2020-08-31,31,12741.47
2020-09-30,30,6351.66
2020-10-31,31,58463.79
2020-11-30,30,7310.79
2020-12-31,31,5005.23
2021-01-31,3220,1086612.71
2021-02-28,3426,1220133.37


In [16]:
print('Last 12 Month Contribution is', round(df_last['Monthly Contribution'].sum(),2))
print('Last 12 Month Orders is', round(df_last['Monthly Orders Count'].sum(),2))

Last 12 Month Contribution is 4044714.97
Last 12 Month Orders is 12391


----

### 5) Average Order Value

In [17]:
# To calculate your company's average order value, simply divide total revenue by the number of orders.
AOV = df['Amount'].sum()/df.shape[0]
print('Average Order Value =', str(round(AOV, 2)))

Average Order Value = 348.93


### 8) Last Order Value

In [18]:
# last registered date
df['Date'].max()

Timestamp('2021-04-30 00:00:00')

In [19]:
# selecting rows based on condition. .. Some issue with timestamps
df4=df[df['Date'] == df['Date'].max()]
df4.tail(3)

,Internal ID,*,Date,Document Number,Company Name,Contact Name,Account,Item,Memo,Amount,Sales Rep,Class,Location,Customer Type by Industry
14696,814255,,2021-04-30,LBAPR210075,Northeastern University - Office of General Co...,Scott Merrill,40200 Sales-E-Discovery,eDiscovery Parent : eDiscovery - Hosting - Rel...,Relativity Users,340.0,Scott Rothschild,EDiscovery/Forensics,Buffalo,Education


In [20]:
print('We have', df4.shape[0], 'orders in the last date')
print('Last registered order value is', df4.iloc[-1]['Amount'])
print("All last day orders total value is {:.2f}".format(df4['Amount'].sum()))

We have 1 orders in the last date
Last registered order value is 340.0
All last day orders total value is 340.00


### 12) Time since last order in days

In [21]:
daysoff=df['Date'].max()-pd.Timestamp.now()
print(abs(daysoff.days), 'days from last order')

41 days from last order


----

### 9) % of Customers that has more value

### 10) % of Orders generated by client

### 11) % of Revenue generated by client

In [22]:
# Let's create separate dataframe with summarized data
df_clients = df.groupby(['Company Name','Customer Type by Industry'])['Amount'].agg(['count','sum']).sort_values(by='sum', ascending=False)
df_clients.rename(columns={'sum':'Total Value', 'count':'Order Count'}, inplace=True)
df_clients['% of Revenue']=df_clients['Total Value']*100/df_clients['Total Value'].sum()
df_clients['% of Orders']=df_clients['Order Count']*100/df_clients['Order Count'].sum()
df_clients

,,Order Count,Total Value,% of Revenue,% of Orders
Company Name,Customer Type by Industry,,,,
"Verisma Systems, Inc",Healthcare,36,403071.69,7.859915,0.244948
McDonald Hopkins (Cleveland),Legal,52,240664.87,4.692975,0.353814
InfoCision,Marketing/Design Agency,8,231890.24,4.521870,0.054433
Elk & Elk,Legal,66,143123.62,2.790917,0.449071
Bonezzi Switzer Polito & Hupp[CLE],Legal,45,130207.00,2.539042,0.306185
...,...,...,...,...,...
Rustbelt,AEC,1,4.20,0.000082,0.006804
NWS Janitorial Services,Retail,1,4.20,0.000082,0.006804
American Steamship Company (ASC),Other,1,2.80,0.000055,0.006804


In [23]:
# If assume what by the "Customers that has more value" we mean customers with bigger "Avg Order Value"
df_clients['Avg order value']=df_clients['Total Value']/df_clients['Order Count']
# from next dataframe we can select group of clients by any special conditions: Top10 by total value, by AOV etc
df_clients.sort_values(by='Avg order value', ascending=False, inplace=True)
df_clients.reset_index(inplace=True)
df_clients

,Company Name,Customer Type by Industry,Order Count,Total Value,% of Revenue,% of Orders,Avg order value
0,InfoCision,Marketing/Design Agency,8,231890.24,4.521870,0.054433,28986.280
1,"Epstein Becker & Green, P.C.",Legal,1,23925.00,0.466539,0.006804,23925.000
2,The Kirtland Country Club,Other,1,20062.50,0.391220,0.006804,20062.500
3,Great Lakes Medical,Healthcare,1,17000.00,0.331501,0.006804,17000.000
4,Opta (USA) Inc.,Manufacturing,3,33900.00,0.661051,0.020412,11300.000
...,...,...,...,...,...,...,...
1557,NWS Janitorial Services,Retail,1,4.20,0.000082,0.006804,4.200
1558,McGuire Development Company,AEC,2,7.95,0.000155,0.013608,3.975
1559,American Steamship Company (ASC),Other,1,2.80,0.000055,0.006804,2.800
1560,Kaz Brothers,AEC,1,2.10,0.000041,0.006804,2.100


In [24]:
df_clients.describe()

,Order Count,Total Value,% of Revenue,% of Orders,Avg order value
count,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000
mean,9.409091,3283.094584,0.064020,0.064020,399.524241
std,25.732692,15665.172313,0.305472,0.175088,1434.042379
min,1.000000,0.000000,0.000000,0.006804,0.000000
25%,1.000000,105.000000,0.002048,0.006804,43.702500
50%,3.000000,381.015000,0.007430,0.020412,106.855606
75%,8.000000,1666.230000,0.032492,0.054433,275.037500
max,489.000000,403071.690000,7.859915,3.327210,28986.280000


In [25]:
# Unique Industry names
df_clients['Customer Type by Industry'].unique()

array(['Marketing/Design Agency', 'Legal', 'Other', 'Healthcare',
       'Manufacturing', 'Business/Consulting Services', 'Tech',
       'Education', 'Banking/Finance', 'Not for Profit', 'Government',
       'AEC', 'Retail'], dtype=object)

In [26]:
# How many clients per Industry
df_clients['Customer Type by Industry'].value_counts()

AEC                             545
Legal                           461
Other                           114
Retail                          107
Not for Profit                   61
Healthcare                       57
Manufacturing                    45
Education                        45
Banking/Finance                  40
Marketing/Design Agency          33
Tech                             22
Government                       18
Business/Consulting Services     14
Name: Customer Type by Industry, dtype: int64

In [27]:
# Set label based on Total Value in one individual Industry, "Legal" for example
x='Legal'
df_cluster=df_clients[(df_clients['Customer Type by Industry'] == x)]
df_cluster.loc[df_cluster['Total Value'] >= df_cluster['Total Value'].quantile(0.75), 'Type'] = 'Best'
df_cluster.loc[(df_cluster['Total Value'] < df_cluster['Total Value'].quantile(0.75))&(df_cluster['Total Value'] > df_cluster['Total Value'].quantile(0.25)), 'Type'] = 'Good'
df_cluster.loc[df_cluster['Total Value'] <= df_cluster['Total Value'].quantile(0.25), 'Type'] = 'Low'   
df_cluster   

,Company Name,Customer Type by Industry,Order Count,Total Value,% of Revenue,% of Orders,Avg order value,Type
1,"Epstein Becker & Green, P.C.",Legal,1,23925.00,0.466539,0.006804,23925.000000,Best
8,Beckage PLLC,Legal,5,37515.70,0.731558,0.034021,7503.140000,Best
12,McGrath North Mullin & Kratz,Legal,5,27421.25,0.534716,0.034021,5484.250000,Best
16,Lamson Dugan & Murray LLP (Omaha),Legal,6,29266.53,0.570699,0.040825,4877.755000,Best
17,McDonald Hopkins (Cleveland),Legal,52,240664.87,4.692975,0.353814,4628.170577,Best
...,...,...,...,...,...,...,...,...
1511,Arlene Bradshaw,Legal,1,10.00,0.000195,0.006804,10.000000,Low
1513,Michael A. Poklar,Legal,2,19.69,0.000384,0.013608,9.845000,Low
1518,Linda M. Campbell,Legal,8,72.97,0.001423,0.054433,9.121250,Low
1539,Bryan Law,Legal,24,161.74,0.003154,0.163299,6.739167,Low


In [28]:
df_cluster['Type'].value_counts()

Good    229
Low     116
Best    116
Name: Type, dtype: int64

#### after clarifying the details of the final look, we can continue with other Types of Industry or other clustering criteria

In [29]:
# Export dataframe for furthermore use
df_clients.to_excel(r'ClientAgregate.xlsx')